In [1]:
import os
import tempfile
import subprocess
import time
from boto.s3.connection import S3Connection
from boto.s3.key import Key
conn = S3Connection("APIKEY", "APIKEYSECRET")

In [14]:
conn.create_bucket('ha2b');

In [309]:
def getDuration(fname):
    # Get frame info
    probeCmd = 'ffprobe -v error -show_format ' + fname
    probe = subprocess.check_output(probeCmd.split())
    matches = filter(lambda s: s.startswith('duration='), probe.split())
    assert len(matches) > 0, "Missing at least one 'duration' key from ffprobe for %s" % fname
    h,s,dur = matches[0].partition('=')
    return float(dur)

In [313]:
def dropFrames(c, k_name, bucket1, bucket2, n_drop_front=3, n_drop_back=3, fps=29.97):
    
    # Fetch video into temp file
    key = c.get_bucket(bucket1).get_key(k_name)
    kname,kExtension = os.path.splitext(k_name)
    f = tempfile.NamedTemporaryFile(delete=False, suffix=kExtension) # Original video
    fName,fExtension = os.path.splitext(f.name)
    h = fName + '2' + fExtension
    key.get_contents_to_file(f)
    f.close()


    # Drop initial frames
    tStart = n_drop_front / fps
    tEnd   = n_drop_back  / fps
    tTotal = getDuration(f.name) - tStart - tEnd
    ffmpegCall = 'ffmpeg -y -ss ' + str(tStart) + ' -t ' + str(tTotal) + ' -i ' + f.name + ' -strict -2 -b:v 2M ' + h
    subprocess.call(ffmpegCall.split())
    
    # Upload result
    k2 = Key(c.get_bucket(bucket2))
    k2.key = k_name
    k2.set_contents_from_filename(h)
    os.unlink(f.name)
    print ("Done: " + k_name)

In [ ]:
for k in b:
    if k.key > '':
        dropFrames(conn,k.key,'ha2','ha2b')
        k.make_public()
    else:
        print("skip")
    

In [316]:
for i in conn.get_bucket('ha2b'):
    i.make_public()